In [1]:
import os 
import json
import vertexai
import base64
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "XXX"
# credentials = service_account.Credentials.from_service_account_file('/Users/giorady/Documents/Work/Datalabs/PythonDev/klg-workshop/key_kawan_lama.json')

In [3]:
vertexai.init(project="datalabs-int-bigdata", location="asia-southeast1")
# vertexai.init(project="gcp-kl-fin-genai-poc", location="asia-southeast1", credentials=credentials)
model = GenerativeModel("gemini-1.5-pro-preview-0409")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 32,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

In [4]:
# File from local machine 
# with open("/Users/giorady/Documents/Work/Datalabs/PythonDev/klg-workshop/pdf_dummy_klg.pdf", "rb") as pdf_file:
#     encoded_string = base64.b64encode(pdf_file.read())

with open("XXX", "rb") as pdf_file:
    encoded_string = base64.b64encode(pdf_file.read())

# File from google cloud storage 
# uri = "gs://kawan_lama_workshop_gio/Order Form Example.pdf"

In [5]:
document = Part.from_data(
mime_type="application/pdf",
data = base64.b64decode(encoded_string)
)

# document = Part.from_uri(
# mime_type="application/pdf",
# uri=uri
# )

In [6]:
prompt = """Extract Reseller Legal Name, Reseller Name, Reseller Address, and the duration term of the products ordered in Year format if its longer or equal to 12 months, from the file and give the output in a format like this without any "json" string indicator in the beginning of your response. Just follow the format:
"reseller_legal_name": "XXXX",
"reseller_name": "XXXX",
"reseller_address": "XXXX",
"term": "XX YEAR",
"""

# prompt = """Extract "KELURAHAN", "KECAMATAN", "KABUPATEN/KOTA", "PROVINSI" and each value from the table "DATA RINCIAN PEROLEHAN SUARA PASANGAN CALON PRESIDEN DAN WAKIL PRESIDEN" from the document provided in a key-value json format like below:
#   {
#    "KELURAHAN": "CILANDAK BARAT"
#    "KECAMATAN": "CILANDAK",
#    "KABUPATEN_KOTA": "KOTA ADM. JAKARTA SELATAN",
#    "PROVINSI": "DKI JAKARTA",
#    "DATA_RINCIAN_PEROIEHAN_SUARA_PASANGAN_CALON_PRESIDEN_DAN_WAKIL_PRESIDEN": {
#     "NOMOR_DAN_NAMA_PASANGAN_CALON": {
#             "ANIES_RASYID_BASWEDAN_MUHAIMIN_ISKANDAR": {
#             "TPS_001": XX,
#             "TPS_002": XX,
#             },
#             "PRABOWO_SUBlANTO_GIBRAN_RAKABUMING_RAKA": {
#             "TPS_001": XX,
#             "TPS_002": XX,
#             },
#             "GANJAR_PRANOWO_MAHFUD_MD": {
#             "TPS_001": XX,
#             "TPS_002": XX,
#             }
#         }
#     }
#   }

# """

# prompt = """Extract "JUMLAH SELURUH SUARA SAH", "JUMLAH SUARA TIDAK SAH", "JUMLAH SELURUH SUARA SAH DAN SUARA TIDAK SAH", from the document provided in a key-value json format like below:
#   {
#    "JUMLAH_SELURUH_SUARA_SAH": {
#    "TPS_1":"XX",
#    "TPS_2":"XX",
#    "TPS_3":"XX",
#    }
#    "JUMLAH_SUARA_TIDAK_SAH": {
#    "TPS_1":"XX",
#    "TPS_2":"XX",
#    "TPS_3":"XX",
#    }
#    "JUMLAH_SELURUH_SUARA_SAH_DAN_TIDAK_SAH": {
#    "TPS_1":"XX",
#    "TPS_2":"XX",
#    "TPS_3":"XX",
#    }
#   }
# """

# prompt = """Extract Reseller Street Address, Reseller Name, and the duration term of the products ordered from the file and give the output in a format like this:
# {
# "street_address": "XXXX",
# "name": "XXXX",
# "term": "XXXX"
# }
# """

In [7]:
responses = model.generate_content(
    [document,prompt],
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [8]:
result = responses.text
print(result)

{
"reseller_legal_name": "PT Data Labs Analytics",
"reseller_name": "Mendi Susanto",
"reseller_address": "Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia",
"term": "1 YEAR"
}


In [ ]:
result_json = json.loads(result)
print(result_json)

{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia', 'name': 'Mendi Susanto', 'term': '12 Months'}


In [ ]:
result_json

{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia',
 'name': 'Mendi Susanto',
 'term': '12 Months'}

In [ ]:
json_rows=[result_json]
print(json_rows)

[{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia', 'name': 'Mendi Susanto', 'term': '12 Months'}]


In [ ]:
json_rows

[{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia',
  'name': 'Mendi Susanto',
  'term': '12 Months'}]

In [ ]:
table_id = "gcp-kl-fin-genai-poc.db_TenantLW.resultTable"
client = bigquery.Client()
job = client.load_table_from_json(json_rows, table_id)
job.result()

LoadJob<project=datalabs-int-bigdata, location=asia-southeast2, id=15b8c619-c746-41fa-ad0d-b546fb01dd9d>